In [0]:
"""
https://www.youtube.com/watch?v=l72hohmWA68
"""

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

schema = StructType([
    StructField("supplier_id", IntegerType()),
    StructField("product_id", IntegerType()),
    StructField("stock_quantity", IntegerType()),
    StructField("record_date", StringType())
]) 

stock = spark.createDataFrame([
    (1, 1, 60, '2022-01-01'),
    (1, 1, 40, '2022-01-02'),
    (1, 1, 35, '2022-01-03'),
    (1, 1, 45, '2022-01-04'),
    (1, 1, 51, '2022-01-06'),
    (1, 1, 55, '2022-01-09'),
    (1, 1, 25, '2022-01-10'),
    (1, 1, 48, '2022-01-11'),
    (1, 1, 45, '2022-01-15'),
    (1, 1, 38, '2022-01-16'),
    (1, 2, 45, '2022-01-08'),
    (1, 2, 40, '2022-01-09'),
    (2, 1, 45, '2022-01-06'),
    (2, 1, 55, '2022-01-07'),
    (2, 2, 45, '2022-01-08'),
    (2, 2, 48, '2022-01-09'),
    (2, 2, 35, '2022-01-10'),
    (2, 2, 52, '2022-01-15'),
    (2, 2, 23, '2022-01-16')
], schema=schema)

stock = stock.withColumn("record_date", col("record_date").cast('date'))

stock.show()
stock.printSchema()

+-----------+----------+--------------+-----------+
|supplier_id|product_id|stock_quantity|record_date|
+-----------+----------+--------------+-----------+
|          1|         1|            60| 2022-01-01|
|          1|         1|            40| 2022-01-02|
|          1|         1|            35| 2022-01-03|
|          1|         1|            45| 2022-01-04|
|          1|         1|            51| 2022-01-06|
|          1|         1|            55| 2022-01-09|
|          1|         1|            25| 2022-01-10|
|          1|         1|            48| 2022-01-11|
|          1|         1|            45| 2022-01-15|
|          1|         1|            38| 2022-01-16|
|          1|         2|            45| 2022-01-08|
|          1|         2|            40| 2022-01-09|
|          2|         1|            45| 2022-01-06|
|          2|         1|            55| 2022-01-07|
|          2|         2|            45| 2022-01-08|
|          2|         2|            48| 2022-01-09|
|          2

In [0]:
stock.filter(col("stock_quantity") <= 50) \
    .drop("stock_quantity") \
    .withColumn("prev_date", lag("record_date").over(Window.partitionBy("supplier_id", "product_id").orderBy("record_date")) ) \
    .withColumn("date_diff", datediff(col("record_date"), col("prev_date"))) \
    .withColumn("grp_flag", when( ((col("date_diff") == 1) | (col("date_diff").isNull())), 0).otherwise(1) ) \
    .withColumn("grp_id", sum("grp_flag").over(Window.partitionBy("supplier_id", "product_id").orderBy("record_date")))\
    .groupBy("supplier_id", "product_id", "grp_id").agg(
        count("*").alias("num_of_records")
        ,min("record_date").alias("start_date")
        ,max("record_date").alias("end_date")
        ) \
    .filter(col("num_of_records") >= 2) \
    .show()

+-----------+----------+------+--------------+----------+----------+
|supplier_id|product_id|grp_id|num_of_records|start_date|  end_date|
+-----------+----------+------+--------------+----------+----------+
|          1|         1|     0|             3|2022-01-02|2022-01-04|
|          1|         1|     1|             2|2022-01-10|2022-01-11|
|          1|         1|     2|             2|2022-01-15|2022-01-16|
|          1|         2|     0|             2|2022-01-08|2022-01-09|
|          2|         2|     0|             3|2022-01-08|2022-01-10|
+-----------+----------+------+--------------+----------+----------+

